In [9]:
# 8/29 FillNA 넣은 것 뺀것

In [1]:
import pandas as pd
import numpy as np
#pyplot import
import matplotlib.pyplot as plt

In [2]:
# activity, payment, label을 불러옴
tr_act=pd.read_csv('lite_data/train_activity_lite.csv')
tr_lab=pd.read_csv('lite_data/train_label_lite.csv')
tr_pay=pd.read_csv('lite_data/train_pay_lite.csv')
te_act=pd.read_csv('lite_data/test_activity_lite.csv')
te_pay=pd.read_csv('lite_data/test_pay_lite.csv')

In [3]:
# 패키지 로드
import pandas as pd
import numpy as np
#pyplot import
import matplotlib.pyplot as plt

In [4]:
# 특정한 column에 대해 8주간의 값을 보여주는 pivot table 생성하는 함수
# 옵션으로 na값을  채울지 말지, 주차별로 확인하고자 하는 값, id를 numeriv으로 줄지를 정할 수 있음
def tms(dat,cname='play_time',replace_text='tr',fill_nas=False,numeric_id=False):
    #activity data의 번호 앞에 tr 혹은 te가 붙어 없애준 열을 하나 만듦
    if numeric_id is True:
        dat1=dat.copy()
        res=dat1.pivot(index='new_id',columns='wk',values=cname)
    else:
        dat1=dat.copy()
        dat1['0']=dat1[dat1.columns[0]].str.replace(replace_text,"").tolist()
        #numeric으로 변환(하지 않으면 정렬이 어려움)
        dat1['0']=pd.to_numeric(dat1['0'])
        #minimum value를 안채운 값 반환
        #index를 기준으로 주차별 column의 값을 나타내는 pivot table 생성
        res=dat1.pivot(index='0',columns='wk',values=cname)
    #fillna가 false면 채우지 않고 리턴
    if fill_nas==False:
        return res
    #True면 채워서 반환
    else:
        res=res.fillna(res.min().min())
        return res
    

In [5]:
#각 변수들이 시간차에 따라 어떻게 변하는지를 plotting 해주는 함수, Na값들을 min 값으로 fill을 함
mapping={'retained':0,'week':1,'month':2,'2month':3}
def tsss1(data,cname='play_time',statistics='mean',mapping=mapping,max_lim=1,numeric_id=False):
    if numeric_id is True:
        dat=data.copy()
        dat=dat.pivot(index='new_id',columns='wk',values=cname)
        dat=dat.fillna(dat.min().min())
    else:
        dat=data.copy()
        dat['0']=dat['new_id'].str.replace('tr',"").tolist()
        dat['0']=pd.to_numeric(dat['0'])
        dat=dat.pivot(index='0',columns='wk',values=cname)
        dat=dat.fillna(dat.min().min())
    for i in ['retained','2month','month','week']:
        temp=dat[lab['label']==i]
        #여기 이하는 신경쓰지 마라
        tagg=temp.agg(statistics,axis=0)
        print(i)
        tagg.index=range(8)
        plt.plot(tagg)
        plt.fill_between(range(8),temp.quantile(q=0.999), temp.quantile(q=0.9), color='k',alpha=.5)
        plt.fill_between(range(8),temp.quantile(q=0.9), temp.quantile(q=0.8), color='y',alpha=.5)
        plt.fill_between(range(8),temp.quantile(q=0.8), temp.quantile(q=0.7), color='g',alpha=.5)
        plt.fill_between(range(8),temp.quantile(q=0.7), temp.quantile(q=0.6), color='b',alpha=.5)
        plt.fill_between(range(8),temp.quantile(q=0.6), temp.quantile(q=0.5), color='c',alpha=.5)
        plt.fill_between(range(8),temp.quantile(q=0.5), temp.quantile(q=0.4), color='m',alpha=.5)
        plt.fill_between(range(8),temp.quantile(q=0.4), temp.quantile(q=0.3), color='w',alpha=.5)
        plt.fill_between(range(8),temp.quantile(q=0.3), temp.quantile(q=0.2), color='r',alpha=.5)
        plt.plot([0,0,0,0,0,0,0,0])

        plt.ylim([data[cname].min(),max_lim])
        plt.show()
    return dat

In [6]:
mapping={'retained':0,'week':1,'month':2,'2month':3}

In [7]:
#act와 payment 합치기 합치는 함수
#payment가 없을 경우 다른 모든 값이 0인 row를 생성
def act_payment_join(act_origin,pay_origin,replace_text='tr',char_replace=False,with_label=False,fill_na=False):
    act=act_origin.copy()
    pay=pay_origin.copy()
    if fill_na==True:
        with_pay=pd.merge(act,pay, how='outer',left_on=['new_id','wk'],right_on=['acc_id','payment_week'])
    else:
        with_pay=pd.merge(act,pay[pay.payment_amount!=pay.payment_amount.min()], how='outer',left_on=['new_id','wk'],right_on=['acc_id','payment_week'])
    with_pay['new_id'][pd.isna(with_pay.new_id)]=with_pay['acc_id'][pd.isna(with_pay.new_id)]
    with_pay['wk'][pd.isna(with_pay.wk)]=with_pay['payment_week'][pd.isna(with_pay.wk)]
    del with_pay['payment_week']
    del with_pay['acc_id']
    if fill_na==True:
        with_pay['cnt_dt'] = with_pay['cnt_dt'].fillna(0)
    #NA값 Min 페딩
    with_pay['payment_amount']=with_pay['payment_amount'].fillna(pay.payment_amount.min())
    for i in with_pay.columns[1:]:
        with_pay[i]=with_pay[i].fillna(with_pay[i].min())
    if char_replace == True:
        with_pay['new_id']=with_pay[with_pay.columns[0]].str.replace(replace_text,"").tolist()
        with_pay['new_id']=pd.to_numeric(with_pay['new_id'])
    if with_label == True:
        with_pay=pd.merge(with_pay,tr_lab)
    return with_pay

In [8]:
#기준이 될 데이터
with_pay_num_id= act_payment_join(tr_act,tr_pay,char_replace=True)
with_pay=act_payment_join(tr_act,tr_pay,char_replace=False)
with_pay.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


,new_id,wk,cnt_dt,play_time,npc_exp,npc_hongmun,quest_exp,quest_hongmun,item_hongmun,game_combat_time,...,normal_chat,whisper_chat,district_chat,party_chat,guild_chat,faction_chat,cnt_use_buffitem,gathering_cnt,making_cnt,payment_amount
0,tr0,8.0,1.0,-0.661470,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354,-0.569842,...,-0.034038,-0.231827,-0.016574,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,-0.149898
1,tr1,8.0,1.0,-0.266961,1.185225,-0.290803,0.251172,-0.487666,-0.306354,-0.309368,...,-0.016354,-0.231827,-0.016574,-0.275174,-0.343293,-0.024028,0.065089,-0.027758,-0.365536,-0.149898
2,tr2,5.0,3.0,1.025391,3.569807,-0.241648,2.333390,1.298843,-0.306354,0.818823,...,0.054378,-0.231827,-0.016574,-0.167158,-0.343293,-0.024028,1.115979,-0.027758,-0.365536,-0.149898
3,tr2,6.0,7.0,3.957989,9.314319,-0.046188,8.192729,6.658372,-0.306354,3.619399,...,0.266577,-0.231827,-0.016574,-0.015936,-0.343293,-0.024028,4.458242,-0.027758,-0.365536,-0.149898
4,tr2,8.0,2.0,0.032587,-0.231874,-0.006354,-0.250423,-0.336445,-0.069520,0.401743,...,-0.034038,-0.108600,-0.016574,-0.080745,-0.343293,-0.024028,0.363021,-0.027758,-0.365536,-0.149898


In [9]:
with_pay.tail()

,new_id,wk,cnt_dt,play_time,npc_exp,npc_hongmun,quest_exp,quest_hongmun,item_hongmun,game_combat_time,...,normal_chat,whisper_chat,district_chat,party_chat,guild_chat,faction_chat,cnt_use_buffitem,gathering_cnt,making_cnt,payment_amount
440573,tr37207,3.0,1.0,-0.661667,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354,-0.569842,...,-0.034038,-0.231827,-0.016574,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,8.744635
440574,tr8876,3.0,1.0,-0.661667,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354,-0.569842,...,-0.034038,-0.231827,-0.016574,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,0.553227
440575,tr35682,3.0,1.0,-0.661667,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354,-0.569842,...,-0.034038,-0.231827,-0.016574,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,0.201664
440576,tr35682,4.0,1.0,-0.661667,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354,-0.569842,...,-0.034038,-0.231827,-0.016574,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,1.607915
440577,tr44843,6.0,1.0,-0.661667,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354,-0.569842,...,-0.034038,-0.231827,-0.016574,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,3.365728


In [10]:
with_pay_te=act_payment_join(te_act,te_pay)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [11]:
len(with_pay)

440578

# payment의 true value 찾기

In [17]:
# paymnet amount를 sorting한 후 그 차이를 확인하면 다음과 같은 값이 나옴,
# 0.00426136485의 배수일 것으로 추정, 이 값을 모든 값에 나눠줌
with_pay_num_id.payment_amount.sort_values().diff().unique()

array([           nan, 0.00000000e+00, 3.51562600e-02, 3.51562600e-02,
       3.51562600e-02, 3.51562600e-02, 3.51562600e-02, 3.51562600e-02,
       4.26136485e-03, 3.08948951e-02, 3.51562600e-02, 3.08948951e-02,
       3.51562600e-02, 4.26136485e-03, 3.08948951e-02, 3.08948951e-02,
       4.26136485e-03, 4.26136485e-03, 3.08948951e-02, 4.26136485e-03,
       4.26136485e-03, 3.08948951e-02, 8.52272969e-03, 2.66335303e-02,
       3.51562600e-02, 3.51562600e-02, 3.51562600e-02, 3.51562600e-02,
       4.26136485e-03, 3.08948951e-02, 3.51562600e-02, 4.26136485e-03,
       4.26136485e-03, 2.66335303e-02, 3.51562600e-02, 3.51562600e-02,
       4.26136485e-03, 3.08948951e-02, 3.51562600e-02, 4.26136485e-03,
       3.08948951e-02, 4.26136485e-03, 3.08948951e-02, 3.51562600e-02,
       3.51562600e-02, 4.26136485e-03, 3.08948951e-02, 8.52272969e-03,
       3.08948951e-02, 4.26136485e-03, 3.08948951e-02, 3.51562600e-02,
       2.66335303e-02, 4.26136485e-03, 3.08948951e-02, 3.51562600e-02,
      

In [18]:
# differnece 들이 0.25의 배수인 것으로 보임
with_pay_num_id.payment_amount.sort_values().diff().unique()/0.0042613648464975284

array([        nan, 0.00000e+00, 8.25000e+00, 8.25000e+00, 8.25000e+00,
       8.25000e+00, 8.25000e+00, 8.25000e+00, 1.00000e+00, 7.25000e+00,
       8.25000e+00, 7.25000e+00, 8.25000e+00, 1.00000e+00, 7.25000e+00,
       7.25000e+00, 1.00000e+00, 1.00000e+00, 7.25000e+00, 1.00000e+00,
       1.00000e+00, 7.25000e+00, 2.00000e+00, 6.25000e+00, 8.25000e+00,
       8.25000e+00, 8.25000e+00, 8.25000e+00, 1.00000e+00, 7.25000e+00,
       8.25000e+00, 1.00000e+00, 1.00000e+00, 6.25000e+00, 8.25000e+00,
       8.25000e+00, 1.00000e+00, 7.25000e+00, 8.25000e+00, 1.00000e+00,
       7.25000e+00, 1.00000e+00, 7.25000e+00, 8.25000e+00, 8.25000e+00,
       1.00000e+00, 7.25000e+00, 2.00000e+00, 7.25000e+00, 1.00000e+00,
       7.25000e+00, 8.25000e+00, 6.25000e+00, 1.00000e+00, 7.25000e+00,
       8.25000e+00, 7.25000e+00, 2.00000e+00, 6.25000e+00, 8.25000e+00,
       7.25000e+00, 6.25000e+00, 8.25000e+00, 1.00000e+00, 7.25000e+00,
       1.00000e+00, 2.00000e+00, 1.00000e+00, 8.25000e+00, 1.000

In [19]:
# payment_amount를 나눠준 후 최소값이 0일 것으로 예상, 이 값을 원래값에 더해봄
with_pay_num_id.payment_amount.sort_values().unique()/0.0042613648464975284

array([-3.51761645e+01, -2.69261645e+01, -1.86761645e+01, -1.04261645e+01,
       -2.17616451e+00,  6.07383549e+00,  1.43238355e+01,  2.25738355e+01,
        3.08238355e+01,  3.90738355e+01,  4.73238355e+01,  5.55738355e+01,
        6.38238355e+01,  6.48238355e+01,  7.20738355e+01,  8.03238355e+01,
        8.13238355e+01,  8.85738355e+01,  9.68238355e+01,  9.78238355e+01,
        1.05073835e+02,  1.06073835e+02,  1.13323835e+02,  1.21573835e+02,
        1.22573835e+02,  1.29823835e+02,  1.30823835e+02,  1.38073835e+02,
        1.39073835e+02,  1.46323835e+02,  1.47323835e+02,  1.54573835e+02,
        1.62823835e+02,  1.64823835e+02,  1.71073835e+02,  1.72073835e+02,
        1.79323835e+02,  1.87573835e+02,  1.88573835e+02,  1.95823835e+02,
        2.04073835e+02,  2.12323835e+02,  2.13323835e+02,  2.20573835e+02,
        2.28823835e+02,  2.29823835e+02,  2.37073835e+02,  2.45323835e+02,
        2.46323835e+02,  2.47323835e+02,  2.53573835e+02,  2.61823835e+02,
        2.70073835e+02,  

In [20]:
#8.25의 배수인 것으로 보임, payment의 최소단위는 990이므로 0 이후 다음 값을 990이 되도록 scaling 해봄
(with_pay_num_id.payment_amount.sort_values().unique()/0.0042613648464975284)-(with_pay_num_id.payment_amount.sort_values().unique()/0.0042613648464975284)[0]

array([   0.        ,    8.25      ,   16.5       ,   24.75      ,
         33.        ,   41.25      ,   49.5       ,   57.75      ,
         66.        ,   74.25      ,   82.5       ,   90.75      ,
         99.        ,  100.        ,  107.25      ,  115.5       ,
        116.5       ,  123.75      ,  132.        ,  133.        ,
        140.25      ,  141.25      ,  148.5       ,  156.75      ,
        157.75      ,  165.        ,  166.        ,  173.25      ,
        174.25      ,  181.5       ,  182.5       ,  189.75      ,
        198.        ,  200.        ,  206.25      ,  207.25      ,
        214.5       ,  222.75      ,  223.75      ,  231.        ,
        239.25      ,  247.5       ,  248.5       ,  255.75      ,
        264.        ,  265.        ,  272.25      ,  280.5       ,
        281.5       ,  282.5       ,  288.75      ,  297.        ,
        305.25      ,  306.25      ,  313.5       ,  321.75      ,
        322.75      ,  330.        ,  331.        ,  338.25   

In [21]:
# 거의 모든 값들이 990의 배수가됨,
# 블소n샵에서 살 수 있는 모든 상품 중 990의 배수가 아닌 상품은 12000원짜리 성별 전환권 밖에 없고 이 값을 제외한 모든 값들이 990의 배수 + 상당히 작은 소수가 됨
((with_pay_num_id.payment_amount.sort_values().unique()/0.0042613648464975284)-(with_pay_num_id.payment_amount.sort_values().unique()/0.0042613648464975284)[0])/8.25*990

array([     0.        ,    990.00000001,   1980.00000002,   2970.00000003,
         3960.00000003,   4950.00000004,   5940.00000005,   6930.00000006,
         7920.00000007,   8910.00000008,   9900.00000009,  10890.00000009,
        11880.0000001 ,  12000.0000001 ,  12870.00000011,  13860.00000012,
        13980.00000012,  14850.00000013,  15840.00000014,  15960.00000014,
        16830.00000014,  16950.00000015,  17820.00000015,  18810.00000016,
        18930.00000016,  19800.00000017,  19920.00000017,  20790.00000018,
        20910.00000018,  21780.00000019,  21900.00000019,  22770.0000002 ,
        23760.0000002 ,  24000.00000021,  24750.00000021,  24870.00000021,
        25740.00000022,  26730.00000023,  26850.00000023,  27720.00000024,
        28710.00000025,  29700.00000026,  29820.00000026,  30690.00000026,
        31680.00000027,  31800.00000027,  32670.00000028,  33660.00000029,
        33780.00000029,  33900.00000029,  34650.0000003 ,  35640.00000031,
        36630.00000031,  

In [90]:
#true value인지 확인
true_payment=((tr_pay.payment_amount/0.0042613648464975284)-(tr_pay.payment_amount.sort_values().unique()/0.0042613648464975284)[0])/8.25*990
(true_payment.astype(int)%990).value_counts()

0      799608
120       372
240        18
480         1
720         1
Name: payment_amount, dtype: int64

In [92]:
(((tr_pay.payment_amount/0.0042613648464975284)-(tr_pay.payment_amount.sort_values().unique()/0.0042613648464975284)[0])/8.25*990).astype(int).value_counts().sort_index()

0         759294
990          454
1980         206
2970         133
3960          95
4950         753
5940          74
6930          31
7920          36
8910          35
9900        2282
10890        111
11880         48
12000         78
12870         32
13860         27
13980          3
14850        288
15840         30
15960          1
16830         16
16950          4
17820          7
18810         22
18930          1
19800       8013
19920          3
20790        163
20910          1
21780         62
           ...  
502920         2
504900        21
509850         1
514800        11
515790         1
517770         1
519750         1
520740         2
524700         4
526680         1
527670         1
544500         3
554400         1
556380         1
564300         3
569250         1
585090         1
586080         1
594000         2
597960         1
602910         1
613800         1
628650         1
652410         1
673200         2
683100         1
712800         1
748440        

In [62]:
# 변환한 값들의 차이를 sorting해봄,
# 모든 값이 990의 배수 혹은 12000의 배수 + 990의 배수임
# 현재 N샵에서 지불 할 수 있는 금액은 990의 배수 이거나 12000원 임, 소수 자리부분은 상당히 작기 때문에 실제 값이라고 볼 수 있음
aa=true_payment.unique()
aa.sort()
aa

array([     0.        ,    990.00000001,   1980.00000002,   2970.00000003,
         3960.00000003,   4950.00000004,   5940.00000005,   6930.00000006,
         7920.00000007,   8910.00000008,   9900.00000009,  10890.00000009,
        11880.0000001 ,  12000.0000001 ,  12870.00000011,  13860.00000012,
        13980.00000012,  14850.00000013,  15840.00000014,  15960.00000014,
        16830.00000014,  16950.00000015,  17820.00000015,  18810.00000016,
        18930.00000016,  19800.00000017,  19920.00000017,  20790.00000018,
        20910.00000018,  21780.00000019,  21900.00000019,  22770.0000002 ,
        23760.0000002 ,  24000.00000021,  24750.00000021,  24870.00000021,
        25740.00000022,  26730.00000023,  26850.00000023,  27720.00000024,
        28710.00000025,  29700.00000026,  29820.00000026,  30690.00000026,
        31680.00000027,  31800.00000027,  32670.00000028,  33660.00000029,
        33780.00000029,  33900.00000029,  34650.0000003 ,  35640.00000031,
        36630.00000031,  

# 실제 값인지를 검증
(실제 값-x)/y = 변환된값 그러므로 실제값 =  y*변환된 값 + x

In [15]:
np.linalg.inv([[1,1],[2,1]])

array([[-1.,  1.],
       [ 2., -1.]])

In [95]:
((np.array([1,2])/0.0042613648464975284)-(tr_pay.payment_amount.sort_values().unique()/0.0042613648464975284)[0])/8.25*990

array([32381.13174429, 60541.12374747])

In [ ]:
위의 값은
x=평균, y= std라 할때
y+x=32381.13174429
2y+x=60541.12374747
라는 연립방정식이 만들어짐

In [96]:
#연립 방정식을 풀면 sd와 평균이 나옴

np.dot(np.linalg.inv([[1,1],[2,1]]),(((np.array([1,2])/0.0042613648464975284)-(tr_pay.payment_amount.sort_values().unique()/0.0042613648464975284)[0])/8.25*990))

array([28159.99200318,  4221.13974111])

In [ ]:
,np.array([32381.13174429, 60541.12374747])

In [97]:
#구한 값과 실제 값의 차이가 있음
print(np.std((true_payment).astype(int)),np.mean((true_payment).astype(int)))

28229.44733338176 4217.692875


In [87]:
te_pay.index

RangeIndex(start=0, stop=320000, step=1)

In [21]:
#test와 train을 합친 다음에 다시 구해봄?

In [22]:
te_pay=pd.read_csv('lite_data/test_pay_lite.csv')
te_act=pd.read_csv('lite_data/test_activity_lite.csv')

In [23]:
col_mins=te_act.min(0)

In [98]:
#test와 tr을 합쳐서 구하니 방정식의 해와 값이 같음
np.mean(true_payment.append(((te_pay['payment_amount']/0.0042613648464975284)-(with_pay_num_id.payment_amount.sort_values().unique()/0.0042613648464975284)[0])/8.25*990))

#Insight 1 :standardization은 test와 tr을 합쳐서 한것으로 간주


4221.1397411077205

In [99]:
true_payment.append(((te_pay['payment_amount']/0.0042613648464975284)-(with_pay_num_id.payment_amount.sort_values().unique()/0.0042613648464975284)[0])/8.25*990).std()

28159.99200317952

## Insight 1 :standardization은 test와 tr을 합쳐서 한것으로 간주

In [102]:
np.mean(((te_pay['payment_amount']/0.0042613648464975284)-(with_pay_num_id.payment_amount.sort_values().unique()/0.0042613648464975284)[0])/8.25*990)

4229.756906286366

In [103]:
with_label_num_id['true_payment']=true_payment

NameError: name 'with_label_num_id' is not defined

In [104]:
true_payment.max()

940500.0000080868

In [106]:
with_pay_num_id.head(30)

,new_id,wk,cnt_dt,play_time,npc_exp,npc_hongmun,quest_exp,quest_hongmun,item_hongmun,game_combat_time,...,normal_chat,whisper_chat,district_chat,party_chat,guild_chat,faction_chat,cnt_use_buffitem,gathering_cnt,making_cnt,payment_amount
0,0,8.0,1.0,-0.661470,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354,-0.569842,...,-0.034038,-0.231827,-0.016574,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,-0.149898
1,1,8.0,1.0,-0.266961,1.185225,-0.290803,0.251172,-0.487666,-0.306354,-0.309368,...,-0.016354,-0.231827,-0.016574,-0.275174,-0.343293,-0.024028,0.065089,-0.027758,-0.365536,-0.149898
2,2,5.0,3.0,1.025391,3.569807,-0.241648,2.333390,1.298843,-0.306354,0.818823,...,0.054378,-0.231827,-0.016574,-0.167158,-0.343293,-0.024028,1.115979,-0.027758,-0.365536,-0.149898
3,2,6.0,7.0,3.957989,9.314319,-0.046188,8.192729,6.658372,-0.306354,3.619399,...,0.266577,-0.231827,-0.016574,-0.015936,-0.343293,-0.024028,4.458242,-0.027758,-0.365536,-0.149898
4,2,8.0,2.0,0.032587,-0.231874,-0.006354,-0.250423,-0.336445,-0.069520,0.401743,...,-0.034038,-0.108600,-0.016574,-0.080745,-0.343293,-0.024028,0.363021,-0.027758,-0.365536,-0.149898
5,3,8.0,2.0,0.044634,1.146086,-0.281333,0.632153,-0.024903,-0.306354,-0.098668,...,0.050842,-0.231827,-0.016574,-0.158517,-0.343293,-0.024028,0.400940,-0.027758,-0.365536,-0.149898
6,4,1.0,7.0,0.745347,-0.222364,1.148897,-0.029679,0.322228,1.895782,0.296181,...,-0.019891,-0.157891,0.020049,1.803049,5.926769,-0.024028,0.395523,-0.027758,-0.365536,6.178228
7,4,2.0,7.0,1.616990,-0.127258,1.978424,-0.139387,1.395226,1.280846,0.838229,...,0.277186,2.134137,0.272558,4.213960,5.033395,-0.024028,0.834297,-0.027758,0.481117,-0.009273
8,4,3.0,7.0,1.378744,-0.231874,0.759673,-0.250423,0.334873,0.628515,0.565298,...,0.118038,0.495214,0.072093,4.702191,3.967375,-0.024028,0.579700,-0.027758,1.327770,2.311040
9,4,4.0,7.0,1.848639,-0.005694,0.523220,0.754450,1.631419,0.435309,0.724192,...,0.362066,0.692378,0.023905,4.684908,3.805690,-0.024028,1.197233,-0.027758,3.867730,-0.149898


In [30]:
#총 지출 금액을 추가함
lab_alpha=lab.copy()
lab_alpha['payment_sum']=with_label_num_id.groupby('new_id')['true_payment'].sum()

In [31]:
ps=lab_alpha['payment_sum'].copy()
ps =ps.sort_values()

In [32]:
np.set_printoptions(precision=5)
np.set_printoptions(suppress=True)

In [33]:
with_pay_num_id[with_pay_num_id['new_id']==3345].iloc[:,:10]

,wk,new_id,payment_amount,cnt_dt,play_time,npc_exp,npc_hongmun,quest_exp,quest_hongmun,item_hongmun
578512,1,3345,17.428232,1.0,-0.466683,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354
578513,2,3345,-0.149898,3.0,-0.556124,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354
578514,3,3345,-0.149898,0.0,-0.661667,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354
578515,4,3345,3.365728,1.0,-0.587062,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354
578516,5,3345,-0.149898,0.0,-0.661667,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354
578517,6,3345,13.912606,3.0,-0.446793,-0.231874,-0.259000,-0.250423,-0.482096,-0.306354
578518,7,3345,-0.149898,0.0,-0.661667,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354
578519,8,3345,7.232916,2.0,-0.588012,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354


In [34]:
with_pay_num_id[with_pay_num_id['new_id']==3345].iloc[:,10:20]

,game_combat_time,get_money,duel_cnt,duel_win,partybattle_cnt,partybattle_win,cnt_enter_inzone_solo,cnt_enter_inzone_light,cnt_enter_inzone_skilled,cnt_enter_inzone_normal
578512,-0.569842,-0.023543,-0.036168,-0.041326,-0.167473,-0.159872,-0.308749,-0.424591,-0.190671,-0.458339
578513,-0.569842,-0.023543,-0.036168,-0.041326,-0.167473,-0.159872,-0.308749,-0.424591,-0.190671,-0.458339
578514,-0.569842,-0.023543,-0.036168,-0.041326,-0.167473,-0.159872,-0.308749,-0.424591,-0.190671,-0.458339
578515,-0.569842,-0.023543,-0.036168,-0.041326,-0.167473,-0.159872,-0.308749,-0.424591,-0.190671,-0.458339
578516,-0.569842,-0.023543,-0.036168,-0.041326,-0.167473,-0.159872,-0.308749,-0.424591,-0.190671,-0.458339
578517,-0.521001,-0.023427,-0.036168,-0.041326,-0.167473,-0.159872,-0.308749,-0.334123,-0.190671,-0.369823
578518,-0.569842,-0.023543,-0.036168,-0.041326,-0.167473,-0.159872,-0.308749,-0.424591,-0.190671,-0.458339
578519,-0.569729,-0.023543,-0.036168,-0.041326,-0.167473,-0.159872,-0.308749,-0.424591,-0.190671,-0.458339


In [35]:
with_pay_num_id[with_pay_num_id['new_id']==3345].iloc[:,20:30]

,cnt_enter_raid,cnt_enter_raid_light,cnt_enter_bam,cnt_clear_inzone_solo,cnt_clear_inzone_light,cnt_clear_inzone_skilled,cnt_clear_inzone_normal,cnt_clear_raid,cnt_clear_raid_light,cnt_clear_bam
578512,-0.36813,-0.402053,-0.047643,-0.306502,-0.430263,-0.188788,-0.462548,-0.367616,-0.401158,-0.03313
578513,-0.36813,-0.402053,-0.047643,-0.306502,-0.430263,-0.188788,-0.462548,-0.367616,-0.401158,-0.03313
578514,-0.36813,-0.402053,-0.047643,-0.306502,-0.430263,-0.188788,-0.462548,-0.367616,-0.401158,-0.03313
578515,-0.36813,-0.402053,-0.047643,-0.306502,-0.430263,-0.188788,-0.462548,-0.367616,-0.401158,-0.03313
578516,-0.36813,-0.402053,-0.047643,-0.306502,-0.430263,-0.188788,-0.462548,-0.367616,-0.401158,-0.03313
578517,-0.36813,-0.402053,-0.047643,-0.306502,-0.378099,-0.188788,-0.411483,-0.367616,-0.401158,-0.03313
578518,-0.36813,-0.402053,-0.047643,-0.306502,-0.430263,-0.188788,-0.462548,-0.367616,-0.401158,-0.03313
578519,-0.36813,-0.402053,-0.047643,-0.306502,-0.430263,-0.188788,-0.462548,-0.367616,-0.401158,-0.03313


In [36]:
with_pay_num_id[with_pay_num_id['new_id']==3345].iloc[:,30:42]

,normal_chat,whisper_chat,district_chat,party_chat,guild_chat,faction_chat,cnt_use_buffitem,gathering_cnt,making_cnt,label,true_payment
578512,-0.026964,-0.096277,-0.016574,-0.257891,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,month,495000.000004
578513,-0.034038,-0.096277,-0.016574,-0.270853,-0.343293,-0.024028,-0.498275,-0.027758,-0.365536,month,0.000000
578514,-0.034038,-0.231827,-0.016574,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,month,0.000000
578515,-0.034038,-0.059309,-0.001154,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,month,99000.000001
578516,-0.034038,-0.231827,-0.016574,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,month,0.000000
578517,-0.034038,0.618441,0.004629,-0.098028,-0.343293,-0.024028,-0.411604,-0.027758,-0.365536,month,396000.000003
578518,-0.034038,-0.231827,-0.016574,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,month,0.000000
578519,-0.019891,-0.022341,-0.016574,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,month,207900.000002


In [37]:
lab_alpha[lab_alpha['payment_sum']>1000000]

,new_id,label,payment_sum
304,tr304,retained,1.049400e+06
592,tr592,retained,1.032570e+06
813,tr813,retained,1.026630e+06
1075,tr1075,retained,1.079100e+06
2618,tr2618,retained,1.202850e+06
3294,tr3294,retained,1.168200e+06
3345,tr3345,month,1.197900e+06
4103,tr4103,2month,1.242450e+06
4966,tr4966,retained,1.079100e+06
5246,tr5246,retained,1.044450e+06


In [38]:
with_pay_num_id[with_pay_num_id['new_id']==98218]['get_money']

648856   -0.021539
648857    0.001591
648858   -0.020669
648859   -0.023444
648860   -0.022555
648861   -0.020287
648862   -0.023166
648863   -0.023077
Name: get_money, dtype: float64

In [62]:
#
with_pay_num_id.head(30)

,wk,new_id,payment_amount,cnt_dt,play_time,npc_exp,npc_hongmun,quest_exp,quest_hongmun,item_hongmun,...,district_chat,party_chat,guild_chat,faction_chat,cnt_use_buffitem,gathering_cnt,making_cnt,label,true_payment,payment_sum
0,1,58254,-0.149898,0.0,-0.661667,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354,...,-0.016574,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,month,0.0,0.000000
1,2,58254,-0.149898,0.0,-0.661667,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354,...,-0.016574,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,month,0.0,0.000000
2,3,58254,-0.149898,0.0,-0.661667,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354,...,-0.016574,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,month,0.0,0.000000
3,4,58254,-0.149898,0.0,-0.661667,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354,...,-0.016574,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,month,0.0,0.000000
4,5,58254,-0.149898,0.0,-0.661667,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354,...,-0.016574,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,month,0.0,407010.000003
5,6,58254,-0.149898,0.0,-0.661667,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354,...,-0.016574,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,month,0.0,0.000000
6,7,58254,-0.149898,4.0,2.088814,4.405057,-0.245477,4.454059,3.009797,-0.306354,...,-0.016574,-0.275174,-0.343293,-0.024028,3.466939,-0.027758,-0.365536,month,0.0,0.000000
7,8,58254,-0.149898,5.0,2.673460,4.760178,-0.222527,6.107479,4.758528,-0.306354,...,-0.016574,-0.275174,-0.343293,-0.024028,5.070359,-0.027758,-0.365536,month,0.0,0.000000
8,1,52531,-0.149898,0.0,-0.661667,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354,...,-0.016574,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,retained,0.0,0.000000
9,2,52531,-0.149898,0.0,-0.661667,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354,...,-0.016574,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,retained,0.0,0.000000


# Play time의 실재 값 복원

In [51]:
#diff 확인
pt=with_label_num_id['play_time'].copy()
uniq_pt=pt.unique()
uniq_pt.sort()
diff_pt=np.diff(uniq_pt)
diff_pt

array([0.00001, 0.00001, 0.00001, ..., 0.00706, 0.00036, 0.17007])

In [55]:
#precision을 더주고 확인
np.set_printoptions(precision=20)
np.set_printoptions(suppress=True)
diff_pt[0:10]

array([0.00000989065203904627, 0.00000989065203904627,
       0.00000989065203793604, 0.00000989065203893524,
       0.00000989065203815809, 0.00000989065203882422,
       0.00000989065203804707, 0.00000989065203904627,
       0.00000989065203893524, 0.00000989065203815809])

In [86]:
(uniq_pt/0.000009890652)

array([-66898.25642292005, -66897.25642291609, -66896.25642291215, ...,
       520670.7458688689 , 520706.74586900923, 537901.7459360778 ])

In [102]:
np.diff(uniq_pt/0.000009890652)

array([    1.000000003958121 ,     1.000000003943569 ,
           1.0000000038271537, ...,   714.0000027854112   ,
          36.00000014033867  , 17195.000067068555    ])

In [104]:
# 동일한 패턴을 나눠 준 후 최소값을 더해주니 minimum 0 ,maximum 604800으로 변환이 됨
# 604800은 7*24*60*60을 나타내고 이는 일주일동안 끄지 않고 게임을 했을시의 playtime을 초로 환산한 값임
#모든 playtime을 초로 환산했다고 생각
# minimum 값이 0인 것이 말아 안된다고 생각 할 수 있지만 정말 0에 가까운 시간을 접속한 유저가 있을 시의 값이라고 생각하였고,
#1~2초 정도의 오차는 감수하는 것이 맞다고 생각하여 제대로 복원하였다고 가정함
(uniq_pt/0.000009890652)-(uniq_pt/0.000009890652).min()

array([     0.               ,      1.000000003958121,
            2.00000000790169 , ..., 587569.0022917889     ,
       587605.0022919293     , 604800.0023589978     ])

604800/60(초->분)/60(분->시간)/24(시간->하루)=7(일주일)
1주일동안 계속 들어와있었던 사람이 있기 때문에 이런식으로 변환하면 초가 나온다고 생각

In [108]:
#위의 값을 실제 값을 추가
with_label_num_id['real_playtime']=((with_label_num_id['play_time']/0.000009890652)-(uniq_pt/0.000009890652).min()).astype(int)
with_label_num_id.head(10)

,wk,new_id,payment_amount,cnt_dt,play_time,npc_exp,npc_hongmun,quest_exp,quest_hongmun,item_hongmun,...,district_chat,party_chat,guild_chat,faction_chat,cnt_use_buffitem,gathering_cnt,making_cnt,label,true_payment,real_playtime
0,1,58254,-0.149898,0.0,-0.661667,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354,...,-0.016574,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,month,0.0,0
1,2,58254,-0.149898,0.0,-0.661667,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354,...,-0.016574,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,month,0.0,0
2,3,58254,-0.149898,0.0,-0.661667,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354,...,-0.016574,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,month,0.0,0
3,4,58254,-0.149898,0.0,-0.661667,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354,...,-0.016574,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,month,0.0,0
4,5,58254,-0.149898,0.0,-0.661667,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354,...,-0.016574,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,month,0.0,0
5,6,58254,-0.149898,0.0,-0.661667,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354,...,-0.016574,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,month,0.0,0
6,7,58254,-0.149898,4.0,2.088814,4.405057,-0.245477,4.454059,3.009797,-0.306354,...,-0.016574,-0.275174,-0.343293,-0.024028,3.466939,-0.027758,-0.365536,month,0.0,278089
7,8,58254,-0.149898,5.0,2.673460,4.760178,-0.222527,6.107479,4.758528,-0.306354,...,-0.016574,-0.275174,-0.343293,-0.024028,5.070359,-0.027758,-0.365536,month,0.0,337200
8,1,52531,-0.149898,0.0,-0.661667,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354,...,-0.016574,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,retained,0.0,0
9,2,52531,-0.149898,0.0,-0.661667,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354,...,-0.016574,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,retained,0.0,0


In [109]:
# 중간 저장
#with_label_num_id.to_csv('label_payment_playtime.csv')

In [ ]:
#불러오기
#with_label_num_id=pd.read_csv('label_payment_playtime.csv')

# 번돈


In [135]:
#diff 확인
gm=with_label_num_id['get_money'].copy()
np.set_printoptions(precision=20)
np.set_printoptions(suppress=True)
uniq_gm=gm.unique()
uniq_gm.sort()
diff_gm=np.diff(uniq_gm)
diff_gm

array([ 0.00000000274640839931,  0.00000000274640849993,
        0.00000000274640849993, ..., 12.794733595397986     ,
       19.556953542628037     ,  1.3711336538749492    ])

In [158]:
np.set_printoptions(precision=5)
np.set_printoptions(suppress=True)
np.array(with_label_num_id['get_money']/min(diff_gm)-(with_label_num_id['get_money']/min(diff_gm)).min())[0:100]

array([       0.     ,        0.     ,        0.     ,        0.     ,
              0.     ,        0.     ,    36538.00099,    44255.0012 ,
              0.     ,        0.     ,        0.     ,        0.     ,
              0.     ,        0.     ,        0.     ,        0.     ,
              0.     ,        0.     ,        0.     ,        0.     ,
              0.     ,        0.     ,        0.     ,        0.     ,
              0.     ,        0.     ,        0.     ,        0.     ,
              0.     ,        0.     ,        0.     ,    20212.00055,
          31076.00084,    27849.00075,    14480.00039,    18432.0005 ,
          43113.00117,    50885.00138,    47125.00128,    42914.00116,
         736987.01996,  1442639.03906,  1196248.03239,   463636.01255,
        1967964.05329, 10601121.28706,  4300833.11646,  7106428.19243,
              0.     ,        0.     ,        0.     ,        0.     ,
              0.     ,        0.     ,        0.     ,        0.     ,
      

In [ ]:
#적당한 값으로 변환하긴 했지만 이를 검증할 방법은 없음, 적당히 보기 좋은 값으로 scale했다고 가정함.

In [161]:
with_label_num_id['expected_gm']=with_label_num_id['get_money']/min(diff_gm)-(with_label_num_id['get_money']/min(diff_gm)).min()
with_label_num_id.head(30)

,wk,new_id,payment_amount,cnt_dt,play_time,npc_exp,npc_hongmun,quest_exp,quest_hongmun,item_hongmun,...,party_chat,guild_chat,faction_chat,cnt_use_buffitem,gathering_cnt,making_cnt,label,true_payment,real_playtime,expected_gm
0,1,58254,-0.149898,0.0,-0.661667,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354,...,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,month,0.0,0,0.000000
1,2,58254,-0.149898,0.0,-0.661667,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354,...,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,month,0.0,0,0.000000
2,3,58254,-0.149898,0.0,-0.661667,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354,...,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,month,0.0,0,0.000000
3,4,58254,-0.149898,0.0,-0.661667,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354,...,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,month,0.0,0,0.000000
4,5,58254,-0.149898,0.0,-0.661667,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354,...,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,month,0.0,0,0.000000
5,6,58254,-0.149898,0.0,-0.661667,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354,...,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,month,0.0,0,0.000000
6,7,58254,-0.149898,4.0,2.088814,4.405057,-0.245477,4.454059,3.009797,-0.306354,...,-0.275174,-0.343293,-0.024028,3.466939,-0.027758,-0.365536,month,0.0,278089,36538.000989
7,8,58254,-0.149898,5.0,2.673460,4.760178,-0.222527,6.107479,4.758528,-0.306354,...,-0.275174,-0.343293,-0.024028,5.070359,-0.027758,-0.365536,month,0.0,337200,44255.001198
8,1,52531,-0.149898,0.0,-0.661667,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354,...,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,retained,0.0,0,0.000000
9,2,52531,-0.149898,0.0,-0.661667,-0.231874,-0.290803,-0.250423,-0.487666,-0.306354,...,-0.275174,-0.343293,-0.024028,-0.503692,-0.027758,-0.365536,retained,0.0,0,0.000000


# 바꿔주는 함수

In [11]:
#원래 값으로 바꿔주는 함수
#위의 변환과정을 참고, cnt에 관한 데이터가 많으므로 sorting 시 가장 작은 차이가 1일것이라는 가정하에 복원한 값을 구할 수 있음
#exp, hongmun 관련 변수들은 제대로 변환했는지 검증할 방법이 없으므로 보기 좋게 scale 했다는 것에 의미를 둠
def to_real_data(dat,cnames,digit_precision=5,depth=1,as_int=False,with_label=False):
    res=[]
    colname_of_res=[]
    np.set_printoptions(precision=digit_precision)
    np.set_printoptions(suppress=True)
    for cname in cnames:
        temp=dat[cname].copy()
        unique_values=temp.unique()
        unique_values.sort()
        for i in range(depth):
            if i == 0:
                diffs=np.diff(unique_values)
            else:
                diffs=np.diff(diffs)
                diffs=diffs[diffs!=0]
            if as_int==True:
                res.append((dat[cname]/min(diffs)-(dat[cname]/min(diffs)).min()).round().astype(int))
            else:
                res.append(dat[cname]/min(diffs)-(dat[cname]/min(diffs)).min())
            if depth == 1:
                colname_of_res.append(cname)
            else:
                colname_of_res.append(cname+'_depth_'+str(i+1))
            if cname=='payment_amount':
                #payment의 경우 최소 단위가 1원이아니므로 다른 scaling이 필요
                res[-1]=(((((((dat['payment_amount']/0.0042613648464975284)-0.0038354911236561564))/0.25)-0.28)*30+4230)+4.255980456946418e-08).astype(int)
    if with_label==True:
        res.append(dat.label)
        colname_of_res.append('label')
        print(dat.label)
    res=np.array(res)
    res=np.transpose(res)
    df_res=pd.DataFrame(res,columns=colname_of_res)
    df_res['cnt_dt']=dat['cnt_dt']
    df_res['wk']=dat['wk']
    df_res['new_id']=dat['new_id']
    return df_res



In [12]:
with_pay_te.columns

Index(['new_id', 'wk', 'cnt_dt', 'play_time', 'npc_exp', 'npc_hongmun',
       'quest_exp', 'quest_hongmun', 'item_hongmun', 'game_combat_time',
       'get_money', 'duel_cnt', 'duel_win', 'partybattle_cnt',
       'partybattle_win', 'cnt_enter_inzone_solo', 'cnt_enter_inzone_light',
       'cnt_enter_inzone_skilled', 'cnt_enter_inzone_normal', 'cnt_enter_raid',
       'cnt_enter_raid_light', 'cnt_enter_bam', 'cnt_clear_inzone_solo',
       'cnt_clear_inzone_light', 'cnt_clear_inzone_skilled',
       'cnt_clear_inzone_normal', 'cnt_clear_raid', 'cnt_clear_raid_light',
       'cnt_clear_bam', 'normal_chat', 'whisper_chat', 'district_chat',
       'party_chat', 'guild_chat', 'faction_chat', 'cnt_use_buffitem',
       'gathering_cnt', 'making_cnt', 'payment_amount'],
      dtype='object')

In [13]:
#NA값 있는지 확인
with_pay.isna().sum()

new_id                      0
wk                          0
cnt_dt                      0
play_time                   0
npc_exp                     0
npc_hongmun                 0
quest_exp                   0
quest_hongmun               0
item_hongmun                0
game_combat_time            0
get_money                   0
duel_cnt                    0
duel_win                    0
partybattle_cnt             0
partybattle_win             0
cnt_enter_inzone_solo       0
cnt_enter_inzone_light      0
cnt_enter_inzone_skilled    0
cnt_enter_inzone_normal     0
cnt_enter_raid              0
cnt_enter_raid_light        0
cnt_enter_bam               0
cnt_clear_inzone_solo       0
cnt_clear_inzone_light      0
cnt_clear_inzone_skilled    0
cnt_clear_inzone_normal     0
cnt_clear_raid              0
cnt_clear_raid_light        0
cnt_clear_bam               0
normal_chat                 0
whisper_chat                0
district_chat               0
party_chat                  0
guild_chat

In [101]:
with_pay_te.columns[2:39]

Index(['cnt_dt', 'play_time', 'npc_exp', 'npc_hongmun', 'quest_exp',
       'quest_hongmun', 'item_hongmun', 'game_combat_time', 'get_money',
       'duel_cnt', 'duel_win', 'partybattle_cnt', 'partybattle_win',
       'cnt_enter_inzone_solo', 'cnt_enter_inzone_light',
       'cnt_enter_inzone_skilled', 'cnt_enter_inzone_normal', 'cnt_enter_raid',
       'cnt_enter_raid_light', 'cnt_enter_bam', 'cnt_clear_inzone_solo',
       'cnt_clear_inzone_light', 'cnt_clear_inzone_skilled',
       'cnt_clear_inzone_normal', 'cnt_clear_raid', 'cnt_clear_raid_light',
       'cnt_clear_bam', 'normal_chat', 'whisper_chat', 'district_chat',
       'party_chat', 'guild_chat', 'faction_chat', 'cnt_use_buffitem',
       'gathering_cnt', 'making_cnt', 'payment_amount'],
      dtype='object')

In [26]:
real_datas=to_real_data(with_pay,with_pay.columns[2:39],depth=1)
real_datas_int=to_real_data(with_pay,with_pay.columns[2:39],depth=1,as_int=True)
real_datas_fillna=to_real_data(act_payment_join(tr_act,tr_pay,fill_na=True),with_pay.columns[2:39],depth=1)
real_datas_fillna_int=to_real_data(act_payment_join(tr_act,tr_pay,fill_na=True),with_pay.columns[2:39],depth=1,as_int=True)
#real_datas_int_num_id=to_real_data(with_label_num_id,with_label_num_id.columns[2:39],depth=1,as_int=True)
#real_datas_int_num_id_label=to_real_data(with_label_num_id,with_label_num_id.columns[2:39],depth=1,as_int=True,with_label=True)
#real_datas_int=to_real_data(with_pay,with_label_num_id.columns[2:39],depth=1,as_int=True)
#real_only_activity=to_real_data(tr_act,tr_act.columns[3:],depth=1)
#real_only_activity_int=to_real_data(tr_act,tr_act.columns[3:],depth=1,as_int=True)
test_real_datas_int=to_real_data(with_pay_te,with_pay_te.columns[2:],as_int=True,depth=1)

In [15]:
real_datas.head(40)

,cnt_dt,play_time,npc_exp,npc_hongmun,quest_exp,quest_hongmun,item_hongmun,game_combat_time,get_money,duel_cnt,...,district_chat,party_chat,guild_chat,faction_chat,cnt_use_buffitem,gathering_cnt,making_cnt,payment_amount,wk,new_id
0,1.0,20.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000e+00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,tr0
1,1.0,39907.000039,46489.000007,0.000000e+00,9.714700e+05,0.000000e+00,0.0,9221.000001,5.437000e+03,0.0,...,0.0,0.0,0.0,0.0,105.0,0.0,0.0,0.0,8.0,tr1
2,3.0,170571.000166,124717.000019,3.909300e+04,5.004230e+06,8.456830e+06,0.0,49160.000007,2.830800e+04,0.0,...,0.0,25.0,0.0,0.0,299.0,0.0,0.0,0.0,5.0,tr2
3,7.0,467073.000455,313170.000047,1.945420e+05,1.635237e+07,3.382732e+07,0.0,148303.000020,1.012040e+05,0.0,...,0.0,60.0,0.0,0.0,916.0,0.0,0.0,0.0,6.0,tr2
4,2.0,70193.000068,0.000000,2.262220e+05,0.000000e+00,7.158400e+05,114.0,34395.000005,3.975700e+04,0.0,...,0.0,45.0,0.0,0.0,160.0,0.0,0.0,0.0,8.0,tr2
5,2.0,71411.000070,45205.000007,7.532000e+03,1.709340e+06,2.190590e+06,0.0,16680.000002,9.343000e+03,0.0,...,0.0,27.0,0.0,0.0,167.0,0.0,0.0,0.0,8.0,tr3
6,7.0,142257.000139,312.000000,1.144989e+06,4.275300e+05,3.833812e+06,1060.0,30658.000004,1.790300e+06,0.0,...,19.0,481.0,2288.0,0.0,166.0,0.0,0.0,178200.0,1.0,tr4
7,7.0,230385.000224,3432.000001,1.804710e+06,2.150500e+05,8.913079e+06,764.0,49847.000007,5.478313e+06,0.0,...,150.0,1039.0,1962.0,0.0,247.0,0.0,7.0,3960.0,2.0,tr4
8,7.0,206297.000201,0.000000,8.354410e+05,0.000000e+00,3.893668e+06,450.0,40185.000005,3.788253e+06,0.0,...,46.0,1152.0,1573.0,0.0,200.0,0.0,14.0,69300.0,3.0,tr4
9,7.0,253806.000247,7420.000001,6.473900e+05,1.946200e+06,1.003115e+07,357.0,45810.000006,4.689469e+06,0.0,...,21.0,1148.0,1514.0,0.0,314.0,0.0,35.0,0.0,4.0,tr4


In [16]:
real_datas.tail()

,cnt_dt,play_time,npc_exp,npc_hongmun,quest_exp,quest_hongmun,item_hongmun,game_combat_time,get_money,duel_cnt,...,district_chat,party_chat,guild_chat,faction_chat,cnt_use_buffitem,gathering_cnt,making_cnt,payment_amount,wk,new_id
440573,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,250470.0,3.0,tr37207
440574,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19800.0,3.0,tr8876
440575,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9900.0,3.0,tr35682
440576,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49500.0,4.0,tr35682
440577,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99000.0,6.0,tr44843


In [108]:
real_datas_int.head()

,payment_amount,cnt_dt,play_time,npc_exp,npc_hongmun,quest_exp,quest_hongmun,item_hongmun,game_combat_time,get_money,...,whisper_chat,district_chat,party_chat,guild_chat,faction_chat,cnt_use_buffitem,gathering_cnt,making_cnt,wk,new_id
0,0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,tr58254
1,0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,tr58254
2,0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,tr58254
3,0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,tr58254
4,0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,tr58254


In [18]:
test_real_datas_int.tail()

,cnt_dt,play_time,npc_exp,npc_hongmun,quest_exp,quest_hongmun,item_hongmun,game_combat_time,get_money,duel_cnt,...,district_chat,party_chat,guild_chat,faction_chat,cnt_use_buffitem,gathering_cnt,making_cnt,payment_amount,wk,new_id
175725,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,158400,5.0,te23623
175726,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,297000,7.0,te23623
175727,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,29700,6.0,te17674
175728,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,50490,4.0,te21469
175729,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,49500,7.0,te21469


In [27]:
#index_수정
real_datas = real_datas.reindex(index = real_datas.new_id.str.replace('tr','').astype(int).sort_values().index)
real_datas_int = real_datas_int.reindex(index = real_datas_int.new_id.str.replace('tr','').astype(int).sort_values().index)
real_datas_fillna = real_datas_fillna.reindex(index = real_datas_fillna.new_id.str.replace('tr','').astype(int).sort_values().index)
real_datas_fillna_int = real_datas_fillna_int.reindex(index = real_datas_fillna_int.new_id.str.replace('tr','').astype(int).sort_values().index)
test_real_datas_int = test_real_datas_int.reindex(index = test_real_datas_int.new_id.str.replace('te','').astype(int).sort_values().index)


In [28]:
test_real_datas_int

,cnt_dt,play_time,npc_exp,npc_hongmun,quest_exp,quest_hongmun,item_hongmun,game_combat_time,get_money,duel_cnt,...,district_chat,party_chat,guild_chat,faction_chat,cnt_use_buffitem,gathering_cnt,making_cnt,payment_amount,wk,new_id
0,4.0,73605,0,585109,0,3275430,568,25310,28722068,3,...,2,221,464,0,122,0,6,118800,1.0,te0
1,5.0,81878,0,557221,0,2512700,490,28832,2716383,3,...,1,387,468,0,199,0,12,58410,2.0,te0
2,4.0,65236,0,495871,0,1838620,998,20016,2384798,2,...,0,112,308,0,108,0,8,149490,3.0,te0
3,4.0,60235,0,273001,0,3350950,1530,19397,181466,3,...,26,218,382,0,110,0,3,495000,4.0,te0
4,4.0,64360,0,355712,2651320,11564720,802,19056,10100765,4,...,0,151,212,0,95,0,5,0,5.0,te0
5,2.0,27039,0,100773,0,1971670,1373,7553,45189,1,...,0,36,115,0,58,0,5,0,6.0,te0
6,3.0,51928,0,259146,0,4032760,868,18399,2443026,2,...,0,102,281,0,80,0,1,0,7.0,te0
7,3.0,51196,0,181801,0,1152700,481,17510,18347838,3,...,0,214,256,0,100,0,13,178200,8.0,te0
14,6.0,145409,802,901195,18498,3000116,530,39010,1655247,0,...,42,687,542,0,46,0,3,257400,7.0,te1
13,7.0,269416,0,1211606,0,4219318,895,41431,5010322,0,...,11,556,704,0,77,0,4,29700,6.0,te1


In [29]:
#저장
real_datas.to_csv('transformed/train_real_values.csv',index=False)
real_datas_int.to_csv('transformed/train_reaal_values_int.csv',index=False)
real_datas_fillna.to_csv('transformed/train_real_values_fillna.csv',index=False)
real_datas_fillna_int.to_csv('transformed/train_real_values_fillna_int.csv',index=False)
#real_only_activity.to_csv('transformed/real_only_activity.csv',index=False)
#real_only_activity_int.to_csv('transformed/real_only_activity_int.csv',index=False)
#test_real_datas_int.to_csv('transformed/test_real_datas_int.csv',index=False)
test_real_datas_int.to_csv('transformed/test_real_datas_int.csv',index=False)